In [1]:
# 查看显卡
!nvidia-smi

Wed Mar 22 09:24:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    33W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#@title 克隆的github仓库
!git clone https://github.com/xzy-git/so-vits-svc -b 4.0


Cloning into 'so-vits-svc'...
remote: Enumerating objects: 531, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 531 (delta 48), reused 37 (delta 37), pack-reused 458
Receiving objects: 100% (531/531), 28.15 MiB | 17.64 MiB/s, done.
Resolving deltas: 100% (220/220), done.


In [3]:
#@title 安装依赖
%cd /content/so-vits-svc
!pip install Flask
!pip install Flask_Cors
!pip install gradio==3.21.0
!pip install numpy==1.20.3
!pip install pyworld==0.2.5
!pip install scipy==1.7.3
!pip install SoundFile==0.12.1
!pip install torch==1.13.1
!pip install torchaudio==0.13.1
!pip install tqdm
!pip install scikit-maad
!pip install praat-parselmouth
!pip install onnx
!pip install onnxsim
!pip install onnxoptimizer
!pip install librosa==0.8.1
!pip install tensorboard

!git clone https://github.com/xzy-git/fairseq
%cd fairseq
!pip install --editable ./
%cd /content/so-vits-svc/


/content/so-vits-svc
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 KB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 25.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 6.8 MB/s eta 0:00

In [4]:
#@title 下载必要模型文件

#@markdown 模型原下载链接：[checkpoint_best_legacy_500.pt](https://ibm.box.com/s/z1wgl1stco8ffooyatzdwsqn2psd9lrr)（源仓库地址：[contentvec](https://github.com/auspicious3000/contentvec)）

!curl -L https://ibm.ent.box.com/shared/static/z1wgl1stco8ffooyatzdwsqn2psd9lrr -o /content/so-vits-svc/hubert/checkpoint_best_legacy_500.pt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    10    0    10    0     0      7      0 --:--:--  0:00:01 --:--:--   178
100 1268M  100 1268M    0     0  10.1M      0  0:02:04  0:02:04 --:--:-- 13.5M


In [6]:
#@title 加载Google云端硬盘，复制模型文件
from google.colab import drive
drive.mount('/content/drive')

!rm -f /content/so-vits-svc/configs/config.json
!cp /content/drive/MyDrive/44k/*.pth  /content/so-vits-svc/logs/44k
!cp /content/drive/MyDrive/44k/*.pt  /content/so-vits-svc/logs/44k
!cp /content/drive/MyDrive/44k/*.json /content/so-vits-svc/configs

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
#@title webui在线推理

#@markdown 设置用户名
user = "altoria"  #@param {type:"string"}
#@markdown 设置密码
password = "xxzzyy123"  #@param {type:"string"}

!python app2.py --user {user} --password {password}

DEBUG:asyncio:Using selector: EpollSelector
DEBUG:asyncio:Using selector: EpollSelector
Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://4bdfc82510942578e8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
DEBUG:multipart.multipart:Calling on_part_begin with no data
DEBUG:multipart.multipart:Calling on_header_field with data[42:61]
DEBUG:multipart.multipart:Calling on_header_value with data[63:89]
DEBUG:multipart.multipart:Calling on_header_end with no data
DEBUG:multipart.multipart:Calling on_headers_finished with no data
DEBUG:multipart.multipart:Calling on_part_data with data[93:100]
DEBUG:multipart.multipart:Calling on_part_end with no data
DEBUG:multipart.multipart:Calling on_part_begin with no data
DEBUG:multipart.multipart:Calling on_header_field with data[144:163]
DEBUG:multipart.multipart:Calling on_header_value with data[165:191]
DEBUG:multipart.multi

In [8]:
#@title 导出onnx供本地推理（colab应该cpu也能跑）

#@markdown **是否导出onnx格式的hubert**
#@markdown <br><br>4.0不再支持自行导出hubert，而是由MoeSS开发者提供onnx<br><br>
#@markdown **是否导出onnx格式的模型**
export_model = True #@param {type:"boolean"}
#@markdown **模型路径**
model_path = "/content/drive/MyDrive/44k" #@param {type:"string"}
#@markdown **工程文件名**
project_name = "Altoria" #@param {type:"string"}
#@markdown **pth模型文件名，如有多个（同一工程，不同step），文件名以逗号分隔**
model_files_input = "G_25600,G_26400" #@param {type:"string"}
model_files = model_files_input.split(',') 

for i in range(len(model_files)):
  !python onnx_export.py --export_model "{export_model}" --model_path "{model_path}" --model_file "{model_files[i]}" --project_name "{project_name}"

2023-03-22 09:47:10 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
True
/content/drive/MyDrive/44k
G_25600
Altoria
load 
2023-03-22 09:47:15 | INFO | root | Loaded checkpoint '/content/drive/MyDrive/44k/G_25600.pth' (iteration 33)
/usr/local/lib/python3.9/dist-packages/torch/onnx/utils.py:2040: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input c
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torch/onnx/utils.py:2040: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input f0
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torch/onnx/utils.py:2040: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input mel2ph
  warnings.warn(
/u